<a href="https://colab.research.google.com/github/gavincapriola/PyTorch-Deep-Learning-and-Artificial-Intelligence/blob/main/Natural%20Language%20Processing%20(NLP)/PyTorch_Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchtext==0.6.0
!pip install spacy==2.2.3
!python -m spacy download en

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 1.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
import torch
import torch.nn as nn
import torchtext.data as ttd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [3]:
data = {
    "label": [0, 1, 1],
    "data": [
        "I like eggs and ham",
        "Eggs I like!",
        "Ham and eggs or just ham?",
    ]
}

In [4]:
df = pd.DataFrame(data)
df.head()

,label,data
0,0,I like eggs and ham
1,1,Eggs I like!
2,1,Ham and eggs or just ham?


In [5]:
df.to_csv("data.csv", index=False)
!head data.csv

label,data
0,I like eggs and ham
1,Eggs I like!
1,Ham and eggs or just ham?


In [6]:
TEXT = ttd.Field(
    sequential=True,
    batch_first=True,
    lower=True,
    tokenize="spacy",
    pad_first=True)
LABEL = ttd.Field(sequential=False, use_vocab=False, is_target=True)

dataset = ttd.TabularDataset(
    path="data.csv",
    format="csv",
    fields=[("label", LABEL), ("data", TEXT),],
    skip_header=True)

In [7]:
ex = dataset.examples[0]
ex.data, ex.label, type(ex)

(['i', 'like', 'eggs', 'and', 'ham'], '0', torchtext.data.example.Example)

In [8]:
train_dataset, test_dataset = dataset.split(0.66)
TEXT.build_vocab(train_dataset,)

In [9]:
vocab = TEXT.vocab
type(vocab), vocab.stoi, vocab.itos

(torchtext.vocab.Vocab,
 defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fb7e9c48890>>,
             {'<unk>': 0,
              '<pad>': 1,
              'eggs': 2,
              'i': 3,
              'like': 4,
              '!': 5,
              'and': 6,
              'ham': 7}),
 ['<unk>', '<pad>', 'eggs', 'i', 'like', '!', 'and', 'ham'])

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [11]:
train_iter, test_iter = ttd.Iterator.splits(
    (train_dataset, test_dataset), sort_key=lambda x: len(x.data),
    batch_sizes=(2, 2), device=device)

In [12]:
for inputs, targets in train_iter:
    print("inputs:", inputs, "shape:", inputs.shape)
    print("targets:", targets, "shape:", targets.shape)
    break

inputs: tensor([[3, 4, 2, 6, 7],
        [1, 2, 3, 4, 5]], device='cuda:0') shape: torch.Size([2, 5])
targets: tensor([0, 1], device='cuda:0') shape: torch.Size([2])


In [13]:
for inputs, targets in test_iter:
    print("inputs:", inputs, "shape:", inputs.shape)
    print("targets:", targets, "shape:", targets.shape)
    break

inputs: tensor([[7, 6, 2, 0, 0, 7, 0]], device='cuda:0') shape: torch.Size([1, 7])
targets: tensor([1], device='cuda:0') shape: torch.Size([1])
